In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys
import numpy as np
import h5netcdf

import xarray as xr
import xarray.ufuncs as xu
import xrft
import pandas as pd
from wrf import smooth2d

from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4, protocol = 'tcp')
client = Client(cluster)
client

/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37285 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:37285/status,
Dashboard: http://127.0.0.1:37285/status,Workers: 1
Total threads: 4,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: tcp://127.0.0.1:43851,Workers: 1
Dashboard: http://127.0.0.1:37285/status,Total threads: 4
Started: Just now,Total memory: 78.61 GiB
Comm: tcp://127.0.0.1:45053,Total threads: 4
Dashboard: http://127.0.0.1:47407/status,Memory: 78.61 GiB
Nanny: None,


In [11]:
indir_data = '/cnrm/tropics/commun/DATACOMMUN/CERES/SYN1deg/CERES_SYN1deg-1H_Terra-Aqua-MODIS_Ed'
ds = xr.open_mfdataset(indir_data + '*.nc', chunks={'time': 1}, parallel=True)
ds

<xarray.Dataset>
Dimensions:      (time: 15608, lat: 180, lon: 360, bnds: 2)
Coordinates:
  * lon          (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat          (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time         (time) float64 3.288e+03 3.288e+03 ... 1.889e+04 1.889e+04
Dimensions without coordinates: bnds
Data variables:
    olr          (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    lon_bounds   (time, lon, bnds) float32 dask.array<chunksize=(365, 360, 2), meta=np.ndarray>
    lat_bounds   (time, lat, bnds) float32 dask.array<chunksize=(365, 180, 2), meta=np.ndarray>
    time_bounds  (time, bnds) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/46)
    conventions:                CF-1.6
    title:                      Daily OLR CDR Product Ver01Rev02
    source:                     NOAA Archive of HIRS L1B data from TIROS-N Se...
    reference:                  doi:10.1175/2007JTECHA989.1  doi:10.1175/1520...
    history:                    2014-06-10T18:57:59Z - generated by software ...
    comment:                    Final TCDR
    ...                         ...
    software_version_id:        Ver01Rev02
    Metadata_Link:              gov.noaa.ncdc:C00875
    product_version:            Ver01Rev02
    platform:                   TIROS-N > Television Infrared Observation Sat...
    sensor:                     HIRS-2 > High Resolution Infra-red Sounder/2,...
    spatial_resolution:         1.0 by 1.0 degree equal angle

In [15]:
ds.time_bounds.attrs['units'] = ds.time.attrs['units']

ds.time_bounds.attrs['calendar'] = ds.time.attrs['calendar']

ds_decoded = xr.decode_cf(ds)
ds_decoded

<xarray.Dataset>
Dimensions:      (time: 15608, lat: 180, lon: 360, bnds: 2)
Coordinates:
  * lon          (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat          (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time         (time) datetime64[ns] 1979-01-01T12:00:00 ... 2021-09-24T12:...
Dimensions without coordinates: bnds
Data variables:
    olr          (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    lon_bounds   (time, lon, bnds) float32 dask.array<chunksize=(365, 360, 2), meta=np.ndarray>
    lat_bounds   (time, lat, bnds) float32 dask.array<chunksize=(365, 180, 2), meta=np.ndarray>
    time_bounds  (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/46)
    conventions:                CF-1.6
    title:                      Daily OLR CDR Product Ver01Rev02
    source:                     NOAA Archive of HIRS L1B data from TIROS-N Se...
    reference:                  doi:10.1175/2007JTECHA989.1  doi:10.1175/1520...
    history:                    2014-06-10T18:57:59Z - generated by software ...
    comment:                    Final TCDR
    ...                         ...
    software_version_id:        Ver01Rev02
    Metadata_Link:              gov.noaa.ncdc:C00875
    product_version:            Ver01Rev02
    platform:                   TIROS-N > Television Infrared Observation Sat...
    sensor:                     HIRS-2 > High Resolution Infra-red Sounder/2,...
    spatial_resolution:         1.0 by 1.0 degree equal angle

In [16]:
ds_decoded.to_netcdf('/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_CLIM/OLR_daily.nc', unlimited_dims='time')